In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras import backend as K

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import io
%matplotlib inline

Using TensorFlow backend.


In [0]:
%cd "/content/drive/My Drive/pubg_Kaggle"
!ls

/content/drive/My Drive/pubg_Kaggle
CNN_weights.h5	pubg2.ipynb  sample_submission_V2.csv  test_V2.csv
pubg1.ipynb	pubg.ipynb   submission.csv	       train_V2.csv


In [0]:
train_df=pd.read_csv("train_V2.csv")
test_df=pd.read_csv("test_V2.csv")
print("Before dropping null values shape of train_df : ", train_df.shape)
train_df=train_df.dropna()
print("After dropping null values shape of train_df : ", train_df.shape)
print("Before dropping null values shape of test_df : ", test_df.shape)
test_df=test_df.dropna()
print("After dropping null values shape of test_df : ", test_df.shape)
print("Columns of train df : ",train_df.columns)
print("Columns of test df : ",test_df.columns)

Before dropping null values shape of train_df :  (4446966, 29)
After dropping null values shape of train_df :  (4446965, 29)
Before dropping null values shape of test_df :  (1934174, 28)
After dropping null values shape of test_df :  (1934174, 28)
Columns of train df :  Index(['Id', 'groupId', 'matchId', 'assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration', 'matchType', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints', 'winPlacePerc'],
      dtype='object')
Columns of test df :  Index(['Id', 'groupId', 'matchId', 'assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration', 'matchType', 'maxPlace',
       'numGroups', 'rankPoi

In [0]:
print("Target Column Description : ",train_df['winPlacePerc'].describe())

Target Column Description :  count    4.446965e+06
mean     4.728216e-01
std      3.074050e-01
min      0.000000e+00
25%      2.000000e-01
50%      4.583000e-01
75%      7.407000e-01
max      1.000000e+00
Name: winPlacePerc, dtype: float64


In [0]:
train_lbl=train_df['winPlacePerc']
train_df.drop(['matchType','rankPoints','winPlacePerc'],axis=1,inplace=True)
test_df.drop(['matchType','rankPoints'],axis=1,inplace=True)

In [0]:
cols_to_drop = ['Id', 'groupId', 'matchId']
features = [col for col in train_df.columns if col not in cols_to_drop]

print("Extracting agg features")

agg_mean = train_df.groupby(['matchId', 'groupId'])[features].mean()
agg_rank = agg_mean.groupby('matchId')[features].rank(pct=True)
agg_median = train_df.groupby(['matchId', 'groupId'])[features].median()

agg_mean.reset_index(inplace=True)
agg_rank.reset_index(inplace=True)
agg_median.reset_index(inplace=True)

print("Merging aff features with train_df")

train_df=train_df.merge(agg_rank,suffixes=['', '_rank'], how='left', on=['matchId', 'groupId'])
train_df=train_df.merge(agg_mean,suffixes=['', '_mean'], how='left', on=['matchId', 'groupId']) 
train_df=train_df.merge(agg_median,suffixes=['', '_median'], how='inner', on=['matchId', 'groupId'])

print("Adding more features")

train_df['healthitems'] = train_df['heals'] + train_df['boosts']
train_df['totalDistance'] = train_df['rideDistance'] + train_df["walkDistance"]+train_df["swimDistance"]
train_df['team_help'] = train_df['revives'] + train_df['assists']
train_df['performance_point'] = train_df['vehicleDestroys'] + train_df['headshotKills'] + train_df['roadKills'] - train_df['teamKills'] + train_df['assists']

print("Dropping Id, groupId, matchId columns")

train_df.drop(['Id','groupId','matchId'],axis=1,inplace=True)



Extracting agg features
Merging aff features with train_df
Adding more features
Dropping Id, groupId, matchId columns


In [0]:
print("Normalizing train_df")

from sklearn import preprocessing
train_df=train_df[:].values
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1), copy=False).fit(train_df)
scaler.transform(train_df)

print("Shape of train_df : ",train_df.shape," , max value of train_df : ", train_df.max()," , min value of train_df : ", train_df.min())

Normalizing train_df
Shape of train_df :  (4446965, 96)  , max value of train_df :  1.0  , min value of train_df :  0.0


In [0]:
from sklearn.model_selection import train_test_split
train_lbl=np.array(train_lbl)
X_train, X_test, y_train, y_test = train_test_split(train_df, train_lbl, test_size=0.3, random_state=42)

print("Shape of X_train : ",X_train.shape)
print("Shape of X_test : ",X_test.shape)

print("Shape of y_train : ",y_train.shape)
print("Shape of y_test : ",y_test.shape)

In [0]:
y_train=y_train.reshape(len(y_train),1)
y_test=y_test.reshape(len(y_test),1)

print("Shape of y_train : ",y_train.shape)
print("Shape of y_test : ",y_test.shape)


In [0]:
from keras.layers import Activation

batch_size = 256
epochs = 11
img_rows, img_cols, channels = 1, 73, 1

if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], channels, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], channels, img_rows, img_cols)
    input_shape = (channels, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, channels)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, channels)
    input_shape = (img_rows, img_cols, channels)
    
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')

In [0]:
model = Sequential()
model.add(Conv2D(4, kernel_size=(1, 3), padding='same', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(8, kernel_size=(1, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(16, kernel_size=(1, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, kernel_size=(1, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, kernel_size=(1, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 2)))
model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))



In [0]:
model.compile(loss=keras.losses.mean_absolute_error, optimizer=keras.optimizers.Adam(), metrics=['mae'])

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test, verbose=2)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
import h5py

model.save_weights('CNN_weights.h5')

In [0]:
model.load_weights('CNN_weights.h5')

In [0]:
print("Extracting agg features")

agg_mean = test_df.groupby(['matchId', 'groupId'])[features].mean()
agg_rank = agg_mean.groupby('matchId')[features].rank(pct=True)

agg_mean.reset_index(inplace=True)
agg_rank.reset_index(inplace=True)

print("Merging agg features with test_df")

test_df=test_df.merge(agg_rank,suffixes=['', '_rank'], how='left', on=['matchId', 'groupId'])
test_df=test_df.merge(agg_mean,suffixes=['', '_mean'], how='left', on=['matchId', 'groupId']) 

print("Adding more features")

test_df['healthitems'] = test_df['heals'] + test_df['boosts']
test_df['totalDistance'] = test_df['rideDistance'] + test_df["walkDistance"]+test_df["swimDistance"]
test_df['team_help'] = test_df['revives'] + test_df['assists']
test_df['performance_point'] = test_df['vehicleDestroys'] + test_df['headshotKills'] + test_df['roadKills'] - test_df['teamKills'] + test_df['assists']

print("Dropping Id, groupId, matchId columns")

test_df.drop(['Id','groupId','matchId'],axis=1,inplace=True)

print("Normalizing test_df")

from sklearn import preprocessing
test_df=test_df[:].values
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1), copy=False).fit(test_df)
scaler.transform(test_df)

print("Shape of test_df : ",test_df.shape," , max value of test_df : ", test_df.max()," , min value of test_df : ", test_df.min())

Extracting agg features
Merging agg features with test_df
Adding more features
Dropping Id, groupId, matchId columns
Normalizing test_df
Shape of test_df :  (1934174, 73)  , max value of test_df :  1.0  , min value of test_df :  0.0


In [0]:
fields = ["Id"]
sub_df = pd.read_csv("test_V2.csv",usecols=fields)



In [0]:
print("Predicting")

if K.image_data_format() == 'channels_first':
    test_df = test_df.reshape(test_df.shape[0], channels, img_rows, img_cols)
    input_shape = (channels, img_rows, img_cols)
else:
    test_df = test_df.reshape(test_df.shape[0], img_rows, img_cols, channels)
    input_shape = (img_rows, img_cols, channels)


pred=model.predict(test_df)
sub_df['winPlacePerc'] = pred
sub_df[["Id", "winPlacePerc"]].to_csv('submission.csv',index=False)

Predicting
